![Image Description](../images/course_logo_2025.png)

![Image Description](../images/ds_venn_en.png)

![Image Description](../images/ds_cycle.png)


![Image Description](../images/python_ds_stack.png)


